In [ ]:
app_ids = [
    "com.bancosol.altoke", 
    "com.bcp.bo.wallet", 
    "bo.com.yolopago", 
    "com.busa.wallet", 
    "com.walletapp.mobile"]

In [ ]:
"""
Script de Recolección Inicial - Apps Bolivia
Ejecutar UNA SOLA VEZ para obtener todo el historial de comentarios
"""

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'scraping_inicial_{datetime.now().strftime("%Y%m%d")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class InitialScraper:
    def __init__(self):
        """Configuración específica para recolección inicial"""
        self.country = 'bo'  
        self.lang = 'es'     
        self.base_delay = (8, 15)  # Delays para recolección masiva
        self.max_retries = 3
        self.output_dir = "../data/initial_reviews"
        self.session_requests = 0
        
    def scrape_app_complete(self, app_id):
        """
        Recolecta TODOS los comentarios históricos de una app
        
        Args:
            app_id (str): ID de la aplicación (ej: 'com.whatsapp')
            
        Returns:
            list: todos los comentarios encontrados
        """
        logger.info(f"🚀 INICIANDO RECOLECCIÓN COMPLETA: {app_id}")
        
        all_reviews = []
        token = None
        page_count = 0
        consecutive_errors = 0
        
        start_time = datetime.now()
        
        while True:
            try:
                # Request a Google Play Store
                result, token = reviews(
                    app_id,
                    lang=self.lang,
                    country=self.country,
                    sort=Sort.NEWEST,  # Más recientes primero
                    count=200,         # Máximo por request
                    continuation_token=token
                )
                
                self.session_requests += 1
                
                # Verificar si hay resultados
                if not result:
                    logger.info(f"📄 Página {page_count}: Sin más comentarios")
                    break
                
                # Validar calidad de datos
                valid_reviews = self._validate_reviews(result, app_id)
                all_reviews.extend(valid_reviews)
                page_count += 1
                
                logger.info(f"📄 Página {page_count}: {len(result)} comentarios | "
                           f"Válidos: {len(valid_reviews)} | Total: {len(all_reviews)}")
                
                # Sin token = fin de datos
                if not token:
                    logger.info("✅ Token agotado - recolección completa")
                    break
                
                consecutive_errors = 0  # Reset contador de errores
                
                # Delay inteligente
                self._smart_delay(page_count)
                
            except Exception as e:
                consecutive_errors += 1
                logger.warning(f"⚠️ Error página {page_count}: {str(e)}")
                
                if consecutive_errors >= self.max_retries:
                    logger.error(f"❌ Máximo de errores alcanzado para {app_id}")
                    break
                
                # Delay por error (exponencial)
                error_delay = min(120, 20 * (2 ** consecutive_errors))
                logger.info(f"😴 Esperando {error_delay}s por error...")
                time.sleep(error_delay)
        
        duration = datetime.now() - start_time
        logger.info(f"🎯 COMPLETADO {app_id}: {len(all_reviews)} comentarios en {duration}")
        
        return all_reviews
    
    def _validate_reviews(self, reviews_batch, app_id):
        """Valida calidad de datos de comentarios"""
        valid_reviews = []
        issues = {'sin_contenido': 0, 'sin_fecha': 0, 'sin_puntuacion': 0}
        
        for review in reviews_batch:
            is_valid = True
            
            # Verificar contenido
            if not review.get('content') or len(review['content'].strip()) < 5:
                issues['sin_contenido'] += 1
                is_valid = False
            
            # Verificar fecha
            if not review.get('at') or not isinstance(review['at'], datetime):
                issues['sin_fecha'] += 1
                is_valid = False
            
            # Verificar puntuación
            if review.get('score') is None or not (1 <= review.get('score', 0) <= 5):
                issues['sin_puntuacion'] += 1
                is_valid = False
            
            if is_valid:
                # Agregar metadatos útiles
                review['app_id'] = app_id
                review['scraped_at'] = datetime.now()
                review['content_length'] = len(review['content'])
                valid_reviews.append(review)
        
        # Log de issues si existen
        if sum(issues.values()) > 0:
            logger.warning(f"⚠️ Issues encontrados: {issues}")
        
        return valid_reviews
    
    def _smart_delay(self, page_count):
        """Sistema de delays inteligente para evitar bloqueos"""
        base_min, base_max = self.base_delay
        
        # Incremento gradual por volumen de requests
        volume_multiplier = 1 + (self.session_requests // 100) * 0.3
        
        # Delays especiales en puntos críticos
        if page_count % 25 == 0:  # Cada 25 páginas (5000 comentarios)
            special_delay = random.uniform(45, 90)
            logger.info(f"🛑 PAUSA ESPECIAL (página {page_count}): {special_delay:.1f}s")
            time.sleep(special_delay)
            return
        
        # Delay normal con variación
        normal_delay = random.uniform(
            base_min * volume_multiplier,
            base_max * volume_multiplier
        )
        
        logger.debug(f"⏳ Delay: {normal_delay:.1f}s")
        time.sleep(normal_delay)
    
    def save_complete_data(self, reviews, app_id):
        """
        Guarda datos completos con metadata rica
        """
        if not reviews:
            logger.warning(f"❌ Sin datos para guardar: {app_id}")
            return None
        
        # Crear directorio
        os.makedirs(self.output_dir, exist_ok=True)
        
        # DataFrame con datos completos
        df = pd.DataFrame(reviews)
        
        # Estadísticas pre-guardado
        stats = self._generate_stats(df, app_id)
        
        # Archivo principal con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        main_file = f"{app_id}_complete_{timestamp}.csv"
        main_path = os.path.join(self.output_dir, main_file)
        
        # Guardar CSV principal
        df.to_csv(main_path, index=False, encoding="utf-8-sig")
        
        # Guardar archivo de estadísticas
        stats_file = f"{app_id}_stats_{timestamp}.txt"
        stats_path = os.path.join(self.output_dir, stats_file)
        
        with open(stats_path, 'w', encoding='utf-8') as f:
            f.write(stats)
        
        logger.info("💾 GUARDADO EXITOSO:")
        logger.info(f"   📁 Datos: {main_file}")
        logger.info(f"   📊 Stats: {stats_file}")
        
        return main_path
    
    def _generate_stats(self, df, app_id):
        """Genera estadísticas detalladas"""
        stats_text = f"""
ESTADÍSTICAS DE RECOLECCIÓN INICIAL
=====================================
App ID: {app_id}
Fecha de scraping: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

VOLUMEN DE DATOS:
• Total comentarios: {len(df):,}
• Rango temporal: {df['at'].min().date()} → {df['at'].max().date()}
• Días cubiertos: {(df['at'].max() - df['at'].min()).days}

DISTRIBUCIÓN DE PUNTUACIONES:
"""
        
        for score in range(1, 6):
            count = len(df[df['score'] == score])
            percentage = (count / len(df)) * 100
            stats_text += f"• {score} estrellas: {count:,} ({percentage:.1f}%)\n"
        
        stats_text += f"""
MÉTRICAS DE CONTENIDO:
• Promedio caracteres: {df['content_length'].mean():.0f}
• Comentario más largo: {df['content_length'].max()} caracteres
• Comentario más corto: {df['content_length'].min()} caracteres

ACTIVIDAD TEMPORAL:
• Comentarios último mes: {len(df[df['at'] > datetime.now() - pd.DateOffset(months=1)]):,}
• Comentarios últimos 7 días: {len(df[df['at'] > datetime.now() - pd.DateOffset(days=7)]):,}
• Puntuación promedio: {df['score'].mean():.2f}/5.0
"""
        
        return stats_text

def main():
    """Función principal para ejecutar recolección inicial"""
    
    # 🔧 CONFIGURAR AQUÍ TUS APP IDs
    # app_ids esta definida una celda arriba
    
    if not app_ids:
        logger.warning("⚠️ CONFIGURAR app_ids en la variable antes de ejecutar")
        print("\n📋 Para usar este script:")
        print("1. Edita la variable 'app_ids' en main()")
        print("2. Agrega los IDs de las apps bolivianas a analizar")
        print("3. Ejecuta: python initial_scraper.py")
        return
    
    scraper = InitialScraper()
    results = {}
    
    logger.info(f"🎯 INICIANDO RECOLECCIÓN INICIAL DE {len(app_ids)} APPS")
    logger.info("📍 País: Bolivia | Idioma: Español")
    
    for i, app_id in enumerate(app_ids, 1):
        logger.info(f"\n{'='*60}")
        logger.info(f"📱 PROCESANDO APP {i}/{len(app_ids)}: {app_id}")
        logger.info(f"{'='*60}")
        
        try:
            # Recolectar todos los comentarios
            reviews = scraper.scrape_app_complete(app_id)
            
            if reviews:
                # Guardar datos
                saved_path = scraper.save_complete_data(reviews, app_id)
                results[app_id] = {
                    'status': 'success',
                    'count': len(reviews),
                    'file': saved_path
                }
                logger.info(f"✅ {app_id}: {len(reviews):,} comentarios guardados")
            else:
                results[app_id] = {'status': 'no_data', 'count': 0, 'file': None}
                logger.warning(f"⚠️ {app_id}: Sin comentarios encontrados")
        
        except Exception as e:
            logger.error(f"❌ ERROR CRÍTICO en {app_id}: {str(e)}")
            results[app_id] = {'status': 'error', 'count': 0, 'error': str(e)}
        
        # Pausa larga entre apps
        if i < len(app_ids):
            inter_app_delay = random.uniform(60, 120)  # 1-2 minutos entre apps
            logger.info(f"🔄 Pausa entre apps: {inter_app_delay:.1f}s")
            time.sleep(inter_app_delay)
    
    # Resumen final
    _print_final_summary(results)

def _print_final_summary(results):
    """Imprime resumen final de la recolección"""
    print(f"\n{'='*80}")
    print("🎉 RECOLECCIÓN INICIAL COMPLETADA")
    print(f"{'='*80}")
    
    total_reviews = sum(r.get('count', 0) for r in results.values())
    successful = sum(1 for r in results.values() if r.get('status') == 'success')
    
    print("📊 RESUMEN GENERAL:")
    print(f"   • Apps procesadas: {len(results)}")
    print(f"   • Apps exitosas: {successful}")
    print(f"   • Total comentarios: {total_reviews:,}")
    
    print("\n📱 DETALLE POR APP:")
    for app_id, result in results.items():
        status_icon = "✅" if result['status'] == 'success' else "❌"
        print(f"   {status_icon} {app_id}: {result.get('count', 0):,} comentarios")
    
    print("\n📁 Archivos guardados en: ../data/initial_reviews/")
    print(f"📝 Log completo en: scraping_inicial_{datetime.now().strftime('%Y%m%d')}.log")

if __name__ == "__main__":
    main()